In [1]:
import numpy as np
import matplotlib.pyplot as plt
import webknossos as wk
from functools import partial
import numpy as np
import skimage
from skimage.future import TrainableSegmenter
from skimage import data, segmentation, feature, future
from sklearn.ensemble import RandomForestClassifier

In [2]:
print(skimage.__version__)

0.19.2


In [3]:
# load annotations
ganglion_segment_anno = wk.Annotation.load('ganglion_segment_data.zip')

In [4]:
# load dataset
dataset = wk.Dataset.open('B002_4x')
layer = dataset.get_layer('versaCT')
CTdata = layer.get_mag("1")

In [5]:
layer_names = [i for i in ganglion_segment_anno.get_volume_layer_names()]
layer_names

['Volume']

In [7]:
ganglion_segment_anno = wk.Annotation.load('ganglion_segment_data.zip')
volume_annotation = ganglion_segment_anno.export_volume_layer_to_dataset(dataset)

In [8]:
volume_annotation = dataset.get_layer('volume_layer')

In [9]:
training_data_bbox = volume_annotation.bounding_box 

In [11]:
features_func = partial(
        feature.multiscale_basic_features, multichannel=False, edges=False
)
segmenter = TrainableSegmenter(features_func=features_func)


In [12]:
img_data_train = CTdata.read(absolute_bounding_box=training_data_bbox)

In [13]:
# (channels, x, y, z)
img_data_train.shape

(1, 192, 256, 288)

In [16]:
X_train = np.moveaxis(np.squeeze(img_data_train), 0, 0).T
X_train.shape

(288, 256, 192)

In [27]:
def stack(_3D_array):
    _2D_stack = []
    for i in _3D_array:
        _2D_stack.append(i)
    _2D_stack = np.vstack(_2D_stack)
    print(_2D_stack.shape)
    return _2D_stack

In [28]:
X_train = stack(X_train)

(73728, 192)


In [20]:
Y_train = np.squeeze(volume_annotation.mags[wk.Mag(1)].read()).T
Y_train.shape

(288, 256, 192)

In [29]:
Y_train = stack(Y_train)

(73728, 192)


In [30]:
segmenter.fit(X_train, Y_train)

C:\Users\bfgra\anaconda3\envs\webknossos\lib\site-packages\skimage\future\trainable_segmentation.py:52: FutureWarning: `multichannel` is a deprecated argument name for `multiscale_basic_features`. It will be removed in version 1.0. Please use `channel_axis` instead.
  self.features = self.features_func(image)


In [31]:
X_predict = np.moveaxis(np.squeeze(CTdata.read()), 0, 0).T
X_predict_y = X_predict.shape[0]

In [32]:
X_predict.shape

(999, 1015, 995)

In [33]:
X_predict = stack(X_predict)

(1013985, 995)


In [34]:
Y_predicted = segmenter.predict(X_predict)

C:\Users\bfgra\anaconda3\envs\webknossos\lib\site-packages\skimage\future\trainable_segmentation.py:85: FutureWarning: `multichannel` is a deprecated argument name for `multiscale_basic_features`. It will be removed in version 1.0. Please use `channel_axis` instead.
  features = self.features_func(image)
